In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import sklearn

## 데이터 로드 및 train test로 분할

In [2]:
df = pd.read_csv('38_Oasis.csv').drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link','target'])



In [3]:
df.head()

,title,date,content
0,[제목 : [오아시스]\n 팬데믹시대에 내마음의 표준을 바꾸는 유통 ],2020-09-04,이번 코로나가 유통시장의 판도를 5년이상 앞당겨진 변화를 가져 왔다?매장에서 인테넷...
1,[제목 : [오아시스] 오아시스 매수 했습니다. ],2021-04-17,"오아이스 주식 중 pe가 보유한 물량이 시장에서 움직이는 듯 하네요.어제, 아시는..."
2,[제목 : [오아시스] 오아시스 매수 했습니다. ],2021-04-17,얼마에 사셨어요? 저도 관심이 있는데요.
3,[제목 : [오아시스]\n 오아시스 주식이요 ],2021-04-17,오아시스 주식을 사고싶어게시판에 올려봅니다파실분 계시면**** 팔육일팔 삼육사일로연...
4,[제목 : [오아시스] 70만원이면 비싼건가요 ],2021-04-19,"30주 70만원에 살려고하는데, 비싼건가요"


## null 값 확인

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    136 non-null    object
 1   date     136 non-null    object
 2   content  133 non-null    object
dtypes: object(3)
memory usage: 3.3+ KB


In [5]:
# NULL 값 있는 행 확인
df[df['content'].isnull()]

,title,date,content
31,[제목 : [오아시스] 상장노력은 하는거죠? ],2021-07-09,NaN
40,[제목 : [오아시스] 조용히 갑시다~~ ],2021-07-16,NaN
43,[제목 : [오아시스] 지금 잘가고 있긴 한데 스피드 좀 올려~~ ],2021-07-29,NaN


In [6]:
# 공백으로 NULL 값 채우기
df = df.fillna('')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    136 non-null    object
 1   date     136 non-null    object
 2   content  136 non-null    object
dtypes: object(3)
memory usage: 3.3+ KB


내용에 빈 값이 있지만 title에서 추출할 것이 있지 않을까 해서 공백으로 채워줌

## index 날짜로 바꿔주기

In [8]:
# date를 오름차순 정렬
df.date = df.date.sort_index(ascending=True)
# index로 설정
df = df.set_index('date')
df

,title,content
date,,
2020-09-04,[제목 : [오아시스]\n 팬데믹시대에 내마음의 표준을 바꾸는 유통 ],이번 코로나가 유통시장의 판도를 5년이상 앞당겨진 변화를 가져 왔다?매장에서 인테넷...
2021-04-17,[제목 : [오아시스] 오아시스 매수 했습니다. ],"오아이스 주식 중 pe가 보유한 물량이 시장에서 움직이는 듯 하네요.어제, 아시는..."
2021-04-17,[제목 : [오아시스] 오아시스 매수 했습니다. ],얼마에 사셨어요? 저도 관심이 있는데요.
2021-04-17,[제목 : [오아시스]\n 오아시스 주식이요 ],오아시스 주식을 사고싶어게시판에 올려봅니다파실분 계시면**** 팔육일팔 삼육사일로연...
2021-04-19,[제목 : [오아시스] 70만원이면 비싼건가요 ],"30주 70만원에 살려고하는데, 비싼건가요"
...,...,...
2022-08-31,[제목 : [오아시스] 우크라이나와 러시아 ],덩치 큰 러시아가 누구다 다 이길거라고 했지만 결과는? 길고 짧은건 대봐야 안다고우...
2022-08-31,"[제목 : [오아시스] 오늘 날짜(2022.08.31) 더벨 기사, 오아시스 기업...",코스닥 상장을 추진 중인 신선식품 새벽배송 기업 오아시스가 기업가치 목표를 최대 2...
2022-09-02,[제목 : [오아시스] 이 시기에 오히려 주목해야 할 기업 ],연일 금리가 급등하고 있다. 매출만을 키우기 위해 무한확장을 거듭했던 스타트업과 유...


## 특수문자 제거 정규표현식

In [9]:
df['title'] = df['title'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')
df['content'] = df['content'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')

/var/folders/62/p7jd9w0n1n3d19j8s3z7k3lw0000gp/T/ipykernel_1541/1046961940.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title'] = df['title'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')
/var/folders/62/p7jd9w0n1n3d19j8s3z7k3lw0000gp/T/ipykernel_1541/1046961940.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = df['content'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')


In [10]:
df

,title,content
date,,
2020-09-04,제목 오아시스\n 팬데믹시대에 내마음의 표준을 바꾸는 유통,이번 코로나가 유통시장의 판도를 5년이상 앞당겨진 변화를 가져 왔다매장에서 인테넷의...
2021-04-17,제목 오아시스 오아시스 매수 했습니다,오아이스 주식 중 pe가 보유한 물량이 시장에서 움직이는 듯 하네요어제 아시는 분...
2021-04-17,제목 오아시스 오아시스 매수 했습니다,얼마에 사셨어요 저도 관심이 있는데요
2021-04-17,제목 오아시스\n 오아시스 주식이요,오아시스 주식을 사고싶어게시판에 올려봅니다파실분 계시면 팔육일팔 삼육사일로연락주세요
2021-04-19,제목 오아시스 70만원이면 비싼건가요,30주 70만원에 살려고하는데 비싼건가요
...,...,...
2022-08-31,제목 오아시스 우크라이나와 러시아,덩치 큰 러시아가 누구다 다 이길거라고 했지만 결과는 길고 짧은건 대봐야 안다고우크...
2022-08-31,제목 오아시스 오늘 날짜20220831 더벨 기사 오아시스 기업가치 최대 2조 제시,코스닥 상장을 추진 중인 신선식품 새벽배송 기업 오아시스가 기업가치 목표를 최대 2...
2022-09-02,제목 오아시스 이 시기에 오히려 주목해야 할 기업,연일 금리가 급등하고 있다 매출만을 키우기 위해 무한확장을 거듭했던 스타트업과 유니...


## 토큰화

In [11]:
okt = Okt()



### 추출했던 word_list에서 불용어들만 정리

In [20]:
import csv

with open("stopword.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(word_list)

In [25]:
with open('stopword.csv', newline='') as f:
    reader = csv.reader(f)
    data = list(reader)

In [27]:
stopword_add = sum(data, [])
stopword_add

['오아시스',
 '내',
 '을',
 '주',
 '식이',
 '요',
 '이면',
 'pe',
 '뭐',
 '에요',
 '위',
 '글',
 '기',
 '때',
 '옆',
 '집',
 '그렇게',
 '일',
 '그',
 '놈',
 '함',
 '알다',
 '1',
 '71',
 '년',
 '안',
 '영',
 '억',
 '임',
 '네',
 '실',
 '137만',
 '가격',
 '다르다',
 '나',
 'K',
 '삼촌',
 '있다',
 '한데',
 '200',
 '얘기',
 '14677',
 '로',
 '이다',
 '에서',
 '입니다',
 '팔',
 '라고',
 '4',
 '대',
 '시',
 '총',
 '평',
 '어떻다',
 '전',
 '3000000',
 '후',
 '액',
 '분과',
 '신',
 '적',
 '주네',
 '수',
 '드리다',
 '요나',
 '분',
 '성은',
 '5',
 '백원',
 '주식',
 '수가',
 '하',
 '지',
 '바로',
 '몇',
 '일이',
 '죠',
 '향',
 '다',
 '업자',
 '만',
 '뭔',
 '건가',
 '기업',
 '벨',
 '류',
 '된거',
 '치다',
 '이상',
 '누구',
 '이름',
 '인가',
 '아시',
 '부터',
 '작',
 '공',
 '모',
 '가가',
 '장',
 '극',
 '40',
 '시간',
 '2',
 '지다',
 '50분',
 '내용',
 '50',
 '배',
 '써다',
 '29일',
 '까지',
 '날짜',
 '4월',
 '저',
 '3',
 '관련',
 '기준',
 '및',
 '52일',
 '조',
 '라',
 '쓰다',
 '어디',
 '어나',
 '오는',
 '즈',
 '아',
 '성',
 '순',
 '익',
 '중',
 '인데',
 '20220831']

In [31]:
# 불용어 처리 

stopwords = stopword_add + ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','\n','\n ', '제목']
stopwords

['오아시스',
 '내',
 '을',
 '주',
 '식이',
 '요',
 '이면',
 'pe',
 '뭐',
 '에요',
 '위',
 '글',
 '기',
 '때',
 '옆',
 '집',
 '그렇게',
 '일',
 '그',
 '놈',
 '함',
 '알다',
 '1',
 '71',
 '년',
 '안',
 '영',
 '억',
 '임',
 '네',
 '실',
 '137만',
 '가격',
 '다르다',
 '나',
 'K',
 '삼촌',
 '있다',
 '한데',
 '200',
 '얘기',
 '14677',
 '로',
 '이다',
 '에서',
 '입니다',
 '팔',
 '라고',
 '4',
 '대',
 '시',
 '총',
 '평',
 '어떻다',
 '전',
 '3000000',
 '후',
 '액',
 '분과',
 '신',
 '적',
 '주네',
 '수',
 '드리다',
 '요나',
 '분',
 '성은',
 '5',
 '백원',
 '주식',
 '수가',
 '하',
 '지',
 '바로',
 '몇',
 '일이',
 '죠',
 '향',
 '다',
 '업자',
 '만',
 '뭔',
 '건가',
 '기업',
 '벨',
 '류',
 '된거',
 '치다',
 '이상',
 '누구',
 '이름',
 '인가',
 '아시',
 '부터',
 '작',
 '공',
 '모',
 '가가',
 '장',
 '극',
 '40',
 '시간',
 '2',
 '지다',
 '50분',
 '내용',
 '50',
 '배',
 '써다',
 '29일',
 '까지',
 '날짜',
 '4월',
 '저',
 '3',
 '관련',
 '기준',
 '및',
 '52일',
 '조',
 '라',
 '쓰다',
 '어디',
 '어나',
 '오는',
 '즈',
 '아',
 '성',
 '순',
 '익',
 '중',
 '인데',
 '20220831',
 '의',
 '가',
 '이',
 '은',
 '들',
 '는',
 '좀',
 '잘',
 '걍',
 '과',
 '도',
 '를',
 '으로',
 '자',
 '에',
 '와',
 '한',
 '하다',


In [32]:
token = []
for sentence in tqdm(df['title']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    token.append(stopwords_removed_sentence)
    

100%|████████████████████████████████████████| 136/136 [00:00<00:00, 518.64it/s]


In [33]:
len(token)

136

In [34]:
from collections import Counter

In [35]:
# 한 날짜? 한 행당 횟수 추출

for i in range(136):
    vocab = Counter(token[i])
    print(vocab)

Counter({'팬데믹': 1, '시대': 1, '마음': 1, '표준': 1, '바꾸다': 1, '유통': 1})
Counter({'매수': 1})
Counter({'매수': 1})
Counter()
Counter({'70만원': 1, '비싸다': 1, '가요': 1})
Counter({'70만원': 1, '비싸다': 1, '가요': 1})
Counter({'코멘트': 1, '가다': 1})
Counter({'망설이다': 1, '못': 1, '사다': 1})
Counter({'바이럴': 1, '막히다': 1})
Counter({'아직': 1, '모르다': 1, '빨리': 1, '사다': 1})
Counter({'비싸다': 2, '너무': 1})
Counter({'코멘트': 1, '그래도': 1, '지금': 1, '제일': 1, '싸다': 1, '겁니다': 1})
Counter({'비싸다': 2, '너무': 1})
Counter({'상장': 1, '언제': 1, '하나요': 1})
Counter({'수수': 1, '수지': 1, '없다': 1})
Counter({'거래': 1, '뜸': 1})
Counter({'코멘트': 1, '싸다': 1, '올리다': 1, '사보다': 1, '거래': 1})
Counter({'수수': 1, '수지': 1, '없다': 1})
Counter({'무상증자': 1})
Counter({'수수': 1, '수지': 1, '없다': 1})
Counter({'백만원': 1, '거래': 1, '되다': 1})
Counter({'매다': 1, '거래': 1, '되다': 1})
Counter({'오늘': 1, '매수': 1})
Counter({'중소기업': 1, '여부': 1})
Counter({'코멘트': 1, '예상': 1, '말고': 1, '올해': 1, '실적': 1, '보다': 1, '2400억': 1, '정도인': 1})
Counter({'사다': 2, '코멘트': 1, '크다': 1, '판돈': 1, '일부': 1, '자다': 1

In [36]:
# 전체 중 빈도수 출력

all_word_list = sum(token, [])
vocab = Counter(all_word_list)
print(vocab)

Counter({'코멘트': 32, '상장': 12, '되다': 9, '1조': 8, '청구': 8, '비싸다': 7, '액면': 7, '분할': 7, '오다': 7, '가다': 6, '사다': 6, '없다': 6, '가치': 6, '지금': 5, '거래': 5, '실적': 5, '보다': 5, '2조': 5, '매수': 4, '너무': 4, '하나요': 4, '가요': 3, '언제': 3, '수수': 3, '수지': 3, '올리다': 3, '유상증자': 3, '공시': 3, '나오다': 3, '투자': 3, '반기': 3, '신청': 3, '컬리': 3, '70만원': 2, '못': 2, '싸다': 2, '겁니다': 2, '무상증자': 2, '백만원': 2, '오늘': 2, '예상': 2, '이미': 2, '제대로': 2, '희망': 2, '무증': 2, 'ipo': 2, '3조': 2, '괜찮다': 2, '어디서': 2, '확인': 2, '빠르다': 2, '고객': 2, '센터': 2, '악의': 2, '매물': 2, '신고': 2, '나다': 2, '적다': 2, '그렇다': 2, '않다': 2, '돈': 2, '내년': 2, '하반기': 2, '들어오다': 2, '안티': 2, '라니': 2, '날아가다': 2, '편지': 2, '문제': 2, '교부': 2, '언급': 2, '에나': 2, '적자': 2, '팬데믹': 1, '시대': 1, '마음': 1, '표준': 1, '바꾸다': 1, '유통': 1, '망설이다': 1, '바이럴': 1, '막히다': 1, '아직': 1, '모르다': 1, '빨리': 1, '그래도': 1, '제일': 1, '뜸': 1, '사보다': 1, '매다': 1, '중소기업': 1, '여부': 1, '말고': 1, '올해': 1, '2400억': 1, '정도인': 1, '크다': 1, '판돈': 1, '일부': 1, '자다': 1, '갈다': 1, '매출': 1, '3천억': 1, '이익': 1, '100억': 1, '남짓':

In [37]:
word_list = list(vocab.keys())
word_list

['팬데믹',
 '시대',
 '마음',
 '표준',
 '바꾸다',
 '유통',
 '매수',
 '70만원',
 '비싸다',
 '가요',
 '코멘트',
 '가다',
 '망설이다',
 '못',
 '사다',
 '바이럴',
 '막히다',
 '아직',
 '모르다',
 '빨리',
 '너무',
 '그래도',
 '지금',
 '제일',
 '싸다',
 '겁니다',
 '상장',
 '언제',
 '하나요',
 '수수',
 '수지',
 '없다',
 '거래',
 '뜸',
 '올리다',
 '사보다',
 '무상증자',
 '백만원',
 '되다',
 '매다',
 '오늘',
 '중소기업',
 '여부',
 '예상',
 '말고',
 '올해',
 '실적',
 '보다',
 '2400억',
 '정도인',
 '크다',
 '판돈',
 '일부',
 '자다',
 '갈다',
 '매출',
 '3천억',
 '이익',
 '100억',
 '남짓',
 '항상',
 '가장',
 '저렴하다',
 '유상증자',
 '공시',
 '나오다',
 '노력',
 '무조건',
 '회',
 '이미',
 '물량많',
 '금액',
 '제대로',
 '성공하다',
 '조용하다',
 '액면',
 '분할',
 '언제나',
 '희망',
 '먹다',
 '스피드',
 '일단',
 '찍다',
 '난후',
 '무증',
 '1조',
 '평가',
 '쏘다',
 '아시아',
 '경제',
 'ib',
 'ipo',
 '가치',
 '5천',
 '3조',
 '계속',
 '연락',
 '오다',
 '글로벌',
 '사모',
 '1천억',
 '들어온답니다',
 '우선주',
 '투자',
 '더',
 '괜찮다',
 '물량',
 '부담',
 '줄다',
 '앞',
 '좋아지다',
 '같다',
 '상',
 '장하다',
 '최소',
 '반기',
 '왜',
 '어디서',
 '확인',
 '자공시',
 '빠르다',
 '쓱',
 '주다',
 '넘기다',
 '식수',
 '청구',
 '신청',
 '고객',
 '센터',
 '악의',
 '매물',
 '신고',
 '나다',
 '따위',
 '답변',
 '차례

In [38]:
word_rank = list(vocab.values())
word_rank

[1,
 1,
 1,
 1,
 1,
 1,
 4,
 2,
 7,
 3,
 32,
 6,
 1,
 2,
 6,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 5,
 1,
 2,
 2,
 12,
 3,
 4,
 3,
 3,
 6,
 5,
 1,
 3,
 1,
 2,
 2,
 9,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 5,
 5,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 3,
 3,
 1,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 7,
 7,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 8,
 1,
 1,
 1,
 1,
 1,
 2,
 6,
 1,
 2,
 1,
 1,
 7,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 3,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 8,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 3,
 1,
 5,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1

In [39]:
data = {'word': word_list, 'count' : word_rank}
word_data = pd.DataFrame.from_dict(data)
word_data

,word,count
0,팬데믹,1
1,시대,1
2,마음,1
3,표준,1
4,바꾸다,1
...,...,...
250,받다,1
251,미트,1
252,박스,1
253,흑자,1


In [41]:
word_data = word_data.sort_values(by=['count'], ascending=False).reset_index().drop(columns=['index'])

In [47]:
for i in range(26):
    print(word_data[ i*10 : (i+1)*10 ])

  word  count
0  코멘트     32
1   상장     12
2   되다      9
3   1조      8
4   청구      8
5   액면      7
6   분할      7
7  비싸다      7
8   오다      7
9   없다      6
   word  count
10   가치      6
11   사다      6
12   가다      6
13   2조      5
14   실적      5
15   보다      5
16   거래      5
17   지금      5
18  하나요      4
19   너무      4
    word  count
20    매수      4
21    투자      3
22    반기      3
23   올리다      3
24  유상증자      3
25    수지      3
26   나오다      3
27    수수      3
28    공시      3
29    언제      3
   word  count
30   신청      3
31   가요      3
32   컬리      3
33  빠르다      2
34  ipo      2
35   내년      2
36   이미      2
37   언급      2
38  제대로      2
39   교부      2
    word  count
40    않다      2
41   그렇다      2
42     돈      2
43    무증      2
44    적다      2
45    문제      2
46    편지      2
47   하반기      2
48    센터      2
49  들어오다      2
    word  count
50    안티      2
51  날아가다      2
52    3조      2
53    라니      2
54    나다      2
55    신고      2
56   괜찮다      2
57    매물      2
58    악의      2
59  

In [48]:
word_data.to_csv('sentimental(38).csv')

In [55]:
sentimental = pd.read_csv('sentimental(38).csv').drop(columns=['Unnamed: 0'])
sentimental.fillna('-')

,word,count,label
0,코멘트,32,-
1,상장,12,-
2,되다,9,-
3,1조,8,-
4,청구,8,-
...,...,...,...
250,같다,1,-
251,상,1,-
252,장하다,1,-
253,최소,1,-
